In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import anndata 
import os
import scanpy as sc 
from matplotlib.gridspec import GridSpec
import quiche as qu
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
adata = anndata.read_h5ad(os.path.join('data', 'Zenodo', 'spain_preprocessed.h5ad'))
adata.X = qu.pp.standardize(adata.X)

In [ ]:
sc.set_figure_params(dpi = 400, dpi_save = 400, fontsize = 14)

colors_dict_cells = {'APC': '#700548',
 'B': '#005377',
 'CAF': '#f2cc8f',
 'CD4T': '#ebb3a9',
 'CD8T': '#ff5666',
 'CD68_Mac': '#ffa52f',
 'CD163_Mac': '#788AA3',
 'Cancer_1': '#66cdaa',
 'Cancer_2': '#3d405b',
 'Cancer_3': '#b49ab8',
 'Endothelium': '#f78e69',
 'Fibroblast': '#2d9bd5',
 'Immune_Other': '#366962',
 'Mac_Other': '#c7d66d',
 'Mast': '#E36414',
 'Monocyte': '#CC6690',
 'NK': '#9ee2ff',
 'Neutrophil': '#4a7c59',
 'Other': '#FFBF69',
 'Smooth_Muscle': '#f5ebe0',
 'T_Other': '#901C14',
 'Treg': '#9e8576'}

phenotypic_markers = ['ECAD', 'CK17', 'CD45', 'CD3', 'CD4', 'CD8', 'FOXP3', 'CD20', 'CD56', 'CD14', 'CD68',
                    'CD163', 'CD11c', 'HLADR', 'ChyTr', 'Calprotectin', 'FAP', 'SMA', 'Vim', 'Fibronectin',
                    'Collagen1', 'CD31']


functional_markers = ['PDL1','Ki67','GLUT1','CD45RO','CD69', 'PD1','CD57','TBET', 'TCF1',
                        'CD45RB', 'TIM3','IDO', 'LAG3', 'CD38']

var_names = phenotypic_markers+functional_markers

cell_ordering = ['Cancer_1', 'Cancer_2', 'Cancer_3', 'CD4T', 'CD8T', 'Treg', 'T_Other', 'B', 
                 'NK', 'CD68_Mac', 'CD163_Mac', 'Mac_Other', 'Monocyte', 'APC','Mast', 'Neutrophil',
                 'CAF', 'Fibroblast', 'Smooth_Muscle', 'Endothelium']

## Supplementary Figure 22

In [ ]:
fig = plt.figure(figsize=(6.25, 5.25), dpi = 400)
gs = GridSpec(1, 2, width_ratios=[1, 1], wspace=0, hspace=0.2, bottom=0.15)  #adjust this depending on how many phenotypic/functional markers you ahve
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
#phenotypic markers
mp1 = sc.pl.matrixplot(adata[adata.obs['Relapse'] == 0], 
                      var_names=functional_markers + ['HLADR'], 
                      groupby='cell_cluster',
                      vmin=-1.5, vmax=2, cmap='vlag', 
                      categories_order=cell_ordering, 
                      ax=ax1, 
                      colorbar_title='avg. expression \n (z-score)', 
                      return_fig=True)

mp1.legend(show = False)
mp1.add_totals(size = 2, color = 'lightgrey', show = False).style(edge_color='black', cmap = 'vlag') #only have this here for edge colors, show = False so we don't see the totals on the first
ax1 = mp1.get_axes()['mainplot_ax']
y_ticks = ax1.get_yticks()
n_norelapse = len(adata[adata.obs['Relapse'] == 0].obs['Patient_ID'].unique())
n_relapse = len(adata[adata.obs['Relapse'] == 1].obs['Patient_ID'].unique())
ax1.set_title(f'No Relapse ($N = {n_norelapse}$)', fontsize = 14)
y_tick_labels = [tick.get_text() for tick in ax1.get_yticklabels()]
ax1.set_yticklabels(y_tick_labels, fontsize=12, ha='right', position=(-0.02, 0))

#cell type circles 
for y, cell_type in zip(y_ticks, y_tick_labels):
    if cell_type in colors_dict_cells:
        color = colors_dict_cells[cell_type]
        ax1.add_patch(plt.Circle((-0.5, y), 0.3, color=color, transform=ax1.transData, clip_on=False, zorder = 10))

line_positions = [3, 9, 16] 
for pos in line_positions:
    ax1.hlines(y=pos, xmin=-0.5, xmax=len(functional_markers)+1, color='black', linewidth=1)

mp2 = sc.pl.matrixplot(adata[adata.obs['Relapse'] == 1], 
                      var_names=functional_markers + ['HLADR'], 
                      groupby='cell_cluster',
                      vmin=-1.5, vmax=2, cmap='vlag', 
                      categories_order=cell_ordering, 
                      ax=ax2, 
                      colorbar_title='avg. expression \n (z-score)', 
                      return_fig=True)
mp2.legend(show = False)
mp2.add_totals(size = 2, color = 'lightgrey', show = False).style(edge_color='black', cmap = 'vlag')
ax2 = mp2.get_axes()['mainplot_ax']
ax2.set_title(f'Relapse ($N = {n_relapse}$)', fontsize = 14)
ax2.set_yticklabels([])
ax2.set_yticks([])

#horizontal lines
for pos in line_positions:
    ax2.hlines(y=pos, xmin=-0.5, xmax=len(functional_markers)+1, color='black', linewidth=1)


plt.savefig(os.path.join('publications','supplementary_figures', 'supplementary_figure22','supplementary_figure22.pdf'), bbox_inches = 'tight')